In [1]:
import torch

In [2]:
from util.data import get_dl, get_train_ds, get_val_ds
from util.training_utils import train, check_accuracy, save, load
from models.Attention3DUNet import Attention3UNet
from util.loss import DiceLoss, BCEDiceLoss

In [3]:
BATCH_SIZE = 4
EPOCHS = 100
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
LR = 0.0001
# DEVICE = 'cpu'

In [5]:
train_dl = get_dl(get_train_ds(), BATCH_SIZE,nw=2)
val_dl = get_dl(get_val_ds(), BATCH_SIZE, nw=2)

images: 221, masks: 221 
images: 148, masks: 148 


In [14]:
import gc
torch.cuda.empty_cache()
gc.collect()
#
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:10000"

In [7]:
model = Attention3UNet(3, 3, features=[64, 128, 256, 512], up_sample=False).to(DEVICE)
print(model)

Attention3UNet(
  (pooling): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (downs): ModuleList(
    (0): DoubleConv3D(
      (conv): Sequential(
        (0): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (1): DoubleConv3D(
      (conv): Sequential(
        (0): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1,

In [8]:
# load(model, "weights/attention/att3d_100e_adam_b4_dice_c")

In [9]:
# model.train()

In [10]:

print(f"total parameters = {sum(p.numel() for p in model.parameters())}") # 5m more params than base3DUnet
print(f"total learnable parameters = {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

total parameters = 91005199
total learnable parameters = 91005199


In [11]:
opt = torch.optim.Adam(model.parameters(), lr=LR)
# loss = torch.nn.BCEWithLogitsLoss # cannot be used because there's a lot of imbalance anyway, so it is better to combine it with dice
loss = BCEDiceLoss()
# loss = DiceLoss()

In [12]:
print(DEVICE)


cuda


In [15]:
train(model, epochs=EPOCHS, training_loader=train_dl, loss_fn=loss, device=DEVICE, optimizer=opt)

  0%|          | 0/56 [00:02<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 8.00 GiB total capacity; 7.16 GiB already allocated; 0 bytes free; 7.26 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
check_accuracy(val_dl, model, DEVICE)
# B3
# 100 DICE Results: Results: 115510440/116391936 with accuracy 99.3187 Dice score: 0.7661410570144653
# 100 BCE-DICE Results: 115626234/116391936 with accuracy 99.3066 Dice score: 0.7499451041221619
# B4
# 100 upsampler DICE Results: Results: 115510440/116391936 with accuracy 99.3227 Dice score: 0.769984781742096
# 100 upsampler BCE-DICE Results: Results: 115510440/116391936 with accuracy A Dice score: D
# 100 conv DICE Results: Results: 115510440/116391936 with accuracy 99.3277 Dice score: 0.7706514596939087
# 100 conv BCE-DICE Results: Results: 115510440/116391936 with accuracy A Dice score: D

In [16]:
# saving sample
save(model,"weights/attention/att3d_100e_adam_b4_bce-dice+10")
print("saved the model...")

saved the model...


In [17]:
train(model, epochs=10, training_loader=train_dl, loss_fn=loss, device=DEVICE, optimizer=opt)

At epoch [1/10]:  76%|███████▌  | 56/74 [01:52<00:36,  2.01s/it, loss=0.0821]


KeyboardInterrupt: 

In [15]:
check_accuracy(val_dl,model,DEVICE)

In [19]:
# loading sample
model1 = Attention3UNet(3,3)
load(model,"weights/attention/t1")